# Convertor for ".mphtxt" to hdf5 format for Matlab

In [ ]:
from exporter import extract_comsol_matrices_from_file, select_matrix_section

### For P2, all important data are here:

In [ ]:
filename_P2 = "data/hetero_paper_corrected_p2.mphtxt"
# Example usage:
sections_P2 = extract_comsol_matrices_from_file(filename_P2)

print("-----------------------------------")
coord_dict_P2 = select_matrix_section(sections_P2, "coordinates", ncol=3)
print(coord_dict_P2["description"])
print(coord_dict_P2["matrix"].shape)

print("-----------------------------------")
elem_dict_P2 = select_matrix_section(sections_P2, "elements", ncol=10)
print(elem_dict_P2["description"])
print(elem_dict_P2["matrix"].shape)

print("-----------------------------------")
elem_tags_dict_P2 = select_matrix_section(sections_P2, "indices", nrow=elem_dict_P2["matrix"].shape[0])
print(elem_tags_dict_P2["description"])
print(elem_tags_dict_P2["matrix"].shape)

print("-----------------------------------")
face_dict_P2 = select_matrix_section(sections_P2, "elements", ncol=6)
print(face_dict_P2["description"])
print(face_dict_P2["matrix"].shape)

print("-----------------------------------")
face_tags_dict_P2 = select_matrix_section(sections_P2, "indices", nrow=face_dict_P2["matrix"].shape[0])
print(face_tags_dict_P2["description"])
print(face_tags_dict_P2["matrix"].shape)

### For P1 (just the mesh), all important data are here:

In [ ]:
filename_P1 = "data/hetero_paper_corrected_p1.mphtxt"
# Example usage:
sections_P1 = extract_comsol_matrices_from_file(filename_P1)

print("-----------------------------------")
coord_dict_P1 = select_matrix_section(sections_P1, "coordinates", ncol=3)
print(coord_dict_P1["description"])
print(coord_dict_P1["matrix"].shape)

print("-----------------------------------")
elem_dict_P1 = select_matrix_section(sections_P1, "elements", ncol=4)
print(elem_dict_P1["description"])
print(elem_dict_P1["matrix"].shape)

print("-----------------------------------")
elem_tags_dict_P1 = select_matrix_section(sections_P1, "indices", nrow=elem_dict_P1["matrix"].shape[0])
print(elem_tags_dict_P1["description"])
print(elem_tags_dict_P1["matrix"].shape)

print("-----------------------------------")
face_dict_P1 = select_matrix_section(sections_P1, "elements", ncol=3)
print(face_dict_P1["description"])
print(face_dict_P1["matrix"].shape)

print("-----------------------------------")
face_tags_dict_P1 = select_matrix_section(sections_P1, "indices", nrow=face_dict_P1["matrix"].shape[0])
print(face_tags_dict_P1["description"])
print(face_tags_dict_P1["matrix"].shape)

### P1 import into Mesh3D object

Post processing taken from previous scripts, and **specific for this problem exports**.

In [ ]:
import numpy as np

# comsol to matlab reordering
reorder_elem = np.array([0, 1, 2, 3, 4, 6, 5, 8, 9, 7])
reorder_face = np.array([0, 1, 2, 3, 5, 4])

# material tags to conform with 4 materials
tags = elem_tags_dict_P2["matrix"]
material = np.zeros_like(tags)
material[tags == 1] = 1
material[tags == 2] = 2
material[tags == 3] = 1
material[tags == 4] = 2
material[tags == 5] = 0
material[tags == 6] = 0
material[tags == 7] = 3
material[tags == 8] = 3

# boundary tags
node_X = coord_dict_P2["matrix"][:, 0]
node_Y = coord_dict_P2["matrix"][:, 1]
node_Z = coord_dict_P2["matrix"][:, 2]
face = face_dict_P2["matrix"]

boundary_outer = np.zeros((face_dict_P2["matrix"].shape[0], 1), dtype=np.int32)
face_X = np.mean(node_X[face], axis=1)
face_Y = np.mean(node_Y[face], axis=1)
face_Z = np.mean(node_Z[face], axis=1)
x_sum = max(node_X)
x_sum_min = min(node_X)
y_sum = max(node_Y)
y_sum_min = min(node_Y)
z_sum = max(node_Z)
z_sum_min = min(node_Z)

boundary_outer[np.isclose(face_X, x_sum_min, atol=1e-2)] = 1
boundary_outer[np.isclose(face_X, x_sum, atol=1e-2)] = 2
boundary_outer[np.isclose(face_Y, y_sum_min, atol=1e-2)] = 3
boundary_outer[np.isclose(face_Y, y_sum, atol=1e-2)] = 4
boundary_outer[np.isclose(face_Z, z_sum_min, atol=1e-2)] = 5
boundary_outer[np.isclose(face_Z, z_sum, atol=1e-2)] = 6

node = coord_dict_P1["matrix"]
face = face_dict_P1["matrix"]
node_PN = coord_dict_P2["matrix"]
face_PN = face_dict_P2["matrix"]
elem_PN = elem_dict_P2["matrix"]

# rorder PN faces and elements
face_PN = face_PN[:, reorder_face]
elem_PN = elem_PN[:, reorder_elem]

#### Export to hdf5


In [ ]:
import h5py
file_name = "test_michalec_opraveny2.h5"

name_all = ['nodeP1', 'node', 'elem', 'material', 'faceP1', 'face', 'boundary']
data_all = [node, node_PN, elem_PN, material, face, face_PN, boundary_outer]

with h5py.File(file_name, 'w') as f:
    for name, data in zip(name_all, data_all):
        f.create_dataset(name=name, data=data, compression='gzip', compression_opts=9)